# Advent of Code 2019

This is a notebook of solutions for the [Advent of Code](http://adventofcode.com) for 2019. Each day will have a link to the problem description. Inputs are stored in the 'data' directory.

## Helper functions

In [2]:
import os
import urllib.request

#adapted from from norvig/pytudes repository
def input(day, year=2019):
    """Open this day's input file."""
    directory = 'input/'
    filename = directory+'day{}.txt'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        if not os.path.exists(directory):
            os.makedirs(directory)

        urllib.request.urlretrieve("https://raw.githubusercontent.com/elahmo/advent-of-code/master/2019/input/" + filename, filename)
        return input(day)

def input_str(day, year=2019): 
    """The contents of this day's input file as a str."""
    return input(day, year).read().rstrip('\n')

def input_list(day, year=2019):
    lines = [line.rstrip('\n') for line in input(day, year)]
    return lines

def input_csv(day, year=2019):
    return [line.split(',') for line in input(day, year)]

def map_tuple(fn, *args): 
    """Do a map, and make the results into a tuple."""
    return tuple(map(fn, *args))

## [Day 1](https://adventofcode.com/2019/day/1): The Tyranny of the Rocket Equation
The first task is to find the fuel requirements for the modules. Fuel required to launch a given module is based on its mass. Specifically, to find the fuel required for a module, take its mass, divide by three, round down, and subtract 2.

In [3]:
modules = input_list(1)
total = sum([int(x)//3-2 for x in modules])

print(f'Total is {total}')

Total is 3454026


### Part Two
Well, the fuel for the modules is calculated, but we need fuel to transfer the fuel too. Interesting!

In [4]:
def fuel_required(mass):
    if mass < 9:
        return 0
    fuel = mass // 3 - 2
    return fuel + fuel_required(fuel)

modules = input_list(1)
total = sum([fuel_required(int(x)) for x in modules])

print(f'Total is {total}')

Total is 5178170


**Comments**:
- good warmup problem, first one is quite simple but it did require me to recall the integer division in Python3, math.floor is an alternative to `//`

## [Day 2](https://adventofcode.com/2019/day/2): 1202 Program Alarm
Today's task is to write a code for a new computer that will be able to parse instructions and perform calculations for gravitation assistance needs. Interesting! 

Instructions are in form of opcode (1 adds, 2 multiplies, 99 error), where addition and multiplication operate on values specified in the next two values.

In [23]:
op_codes = [int(x) for x in input_csv(2)[0]]

In [6]:
op_codes = [int(x) for x in input_csv(2)[0]]
op_codes[1] = 12
op_codes[2] = 2
def run_instructions(op_codes):
    idx = 0
    while True:
        op, a, b, res = op_codes[idx:idx+4]
        if op == 1:
            op_codes[res] = op_codes[a] + op_codes[b]
        elif op == 2:
            op_codes[res] = op_codes[a] * op_codes[b]
        elif op == 99:
            break
        idx += 4
    return op_codes[0]
run_instructions(op_codes)

2890696

### Part Two

In [7]:
memory = [int(x) for x in input_csv(2)[0]]
for x in range(0,100):
    for y in range(0,100):
        ops = memory.copy()
        ops[1] = x
        ops[2] = y
        if run_instructions(ops) == 19690720:
            print(x,y)

82 26


## [Day 3](https://adventofcode.com/2019/day/3): Crossed Wires
Goal is to find the closest intersection between wires (using Manhattan distance). Input is the list of lengths in each directions wires take.

In [83]:
wire_one = [x for x in input_csv(3)[0]]
wire_two = [x for x in input_csv(3)[1]]

In [90]:
# contains the instructions for movements in the grid based on the instruction
mappings = {
    'R': (0, 1),
    'L': (0, -1),
    'U': (1, 0),
    'D': (-1, 0),
}
def populate_grid_simple(instructions):
    wire_grid = {}
    x, y = 0, 0
    for instruction in instructions:
        d = instruction[0]
        length = int(instruction[1:])
        for i in range(length):
            wire_grid[(x, y)] = 1
            x += mappings[d][0]
            y += mappings[d][1]
    return wire_grid

grid_one_simple = populate_grid_simple(wire_one)
grid_two_simple = populate_grid_simple(wire_two)

In [88]:
from collections import Counter
main_grid = Counter(grid_one_simple) + Counter(grid_two_simple)
overlaps = [k for k,v in main_grid.items() if v == 2]

def manhattan_distance(a, b):
    return abs(a) + abs(b)

lowest_distance = float('inf')
lowest_overlap = None
for overlap in overlaps[1:]:
    if manhattan_distance(*overlap) < lowest_distance:
        lowest_distance = manhattan_distance(*overlap)
        lowest_overlap = overlap
lowest_overlap

(0, 227)

### Part 2
Now for distance, use the number of steps, rather than Manhattan distance.

Instead of overcomplicating things in the initial function, I will just use a counter and store the value for the matching points, and use those to find the lowest amount.

In [85]:
def populate_grid(instructions):
    wire_grid = {}
    x, y, steps = 0, 0, 0
    for instruction in instructions:
        d = instruction[0]
        length = int(instruction[1:])
        for i in range(length):
            current_steps = wire_grid.get((x, y), 0) or steps
            wire_grid[(x, y)] = [1, current_steps]
            x += mappings[d][0]
            y += mappings[d][1]
            steps += 1
    return wire_grid
grid_one = populate_grid(wire_one)
grid_two = populate_grid(wire_two)

In [86]:
from collections import Counter
main_grid = Counter(grid_one_simple) + Counter(grid_two_simple)
overlaps = [k for k,v in main_grid.items() if v == 2]

In [89]:
lowest_steps = float('inf')
lowest_overlap = None
for overlap in overlaps[1:]:
    steps = grid_one[overlap][1] + grid_two[overlap][1]
    if steps < lowest_steps:
        lowest_steps = steps
        lowest_overlap = overlap
lowest_steps

20286

## [Day 4](https://adventofcode.com/2019/day/4): Secure container
Time for some password hacking. For a given range, and a set of rules, find the number of possible password combinations.

In [108]:
from collections import Counter

def get_digit(number, n):
    return number // 10**n % 10

def has_two(num):
    return (
        get_digit(num, 5) == get_digit(num, 4)
        or get_digit(num, 4) == get_digit(num, 3)
        or get_digit(num, 3) == get_digit(num, 2)
        or get_digit(num, 2) == get_digit(num, 1)
        or get_digit(num, 1) == get_digit(num, 0)
    ) and (
        # part two modification
        2 in Counter(str(num)).values()
    )

def is_increasing(num):
    return (
        get_digit(num, 5) <= get_digit(num, 4)
        and get_digit(num, 4) <= get_digit(num, 3)
        and get_digit(num, 3) <= get_digit(num, 2)
        and get_digit(num, 2) <= get_digit(num, 1)
        and get_digit(num, 1) <= get_digit(num, 0)
    )

low, high = 240298, 784956

possible_pass = []

for x in range(low, high):
    if has_two(x) and is_increasing(x):
        possible_pass.append(x)

In [109]:
len(possible_pass)

748